## Business Problem

This dataset represents customer information and purchase activity from an e-commerce platform. A business would typically use this data to understand how customers behave, which products perform well, and how sales change over time.

Currently, the data cannot be reliably used for analysis due to potential missing details, duplicates, or inconsistencies. This notebook validates and cleans the data to ensure accurate downstream reporting.

## Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import os

## Phase 1: Data Ingestion

Load both datasets to begin the assessment process.

### Load Customer Details

In [2]:
customer_df = pd.read_csv('../data/customer_details.csv')
print("Customer Details Dataset")
print(f"Shape: {customer_df.shape}")
customer_df.head()

Customer Details Dataset
Shape: (20000, 4)


,customer_id,sex,customer_age,tenure
0,9798859,Male,44.0,93
1,11413563,Male,36.0,65
2,818195,Male,35.0,129
3,12049009,Male,33.0,58
4,10083045,Male,42.0,88


### Load Basket Details

In [3]:
basket_df = pd.read_csv('../data/basket_details.csv')
print("Basket Details Dataset")
print(f"Shape: {basket_df.shape}")
basket_df.head()

Basket Details Dataset
Shape: (15000, 4)


,customer_id,product_id,basket_date,basket_count
0,42366585,41475073,2019-06-19,2
1,35956841,43279538,2019-06-19,2
2,26139578,31715598,2019-06-19,3
3,3262253,47880260,2019-06-19,2
4,20056678,44747002,2019-06-19,2


## Phase 2: Initial Assessment (EDA)

Before cleaning the data, we need to understand the quality issues and document them systematically.

### Initial Data Quality Concerns

Based on an initial review of the datasets, there are a few potential data quality issues that need to be explored before analysis:

- Some customer records may be missing key information such as age or tenure, which could impact customer-level analysis.
- There may be duplicate customer or basket entries that could inflate transaction counts if not handled correctly.
- The `basket_date` field should be checked to ensure dates are consistently formatted and usable for time-based analysis.
- The meaning of `basket_count` is not immediately clear and may represent repeated events rather than unique purchases, which could lead to misinterpretation.
- There may be basket records that do not properly match with customer records, which could cause issues when joining the datasets.

These points will be validated during the data cleaning process before any reporting or insights are generated.

### Customer Details: General Inspection

In [4]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   20000 non-null  int64  
 1   sex           20000 non-null  object 
 2   customer_age  20000 non-null  float64
 3   tenure        20000 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 625.1+ KB


In [5]:
customer_df.describe()

,customer_id,customer_age,tenure
count,2.000000e+04,20000.000000,20000.000000
mean,1.760040e+07,262.222550,44.396800
std,8.679505e+06,604.321589,31.998376
min,2.093000e+03,-34.000000,4.000000
25%,1.188115e+07,29.000000,21.000000
50%,1.560912e+07,38.000000,35.000000
75%,2.228484e+07,123.000000,60.000000
max,4.462566e+07,2022.000000,133.000000


The data is mostly clean. However, the mean of the age seems off here. A mean of 262 is not normal and this might refer to data entry issues which would need further inspections.

### Basket Details: General Inspection

In [6]:
basket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   15000 non-null  int64 
 1   product_id    15000 non-null  int64 
 2   basket_date   15000 non-null  object
 3   basket_count  15000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 468.9+ KB


In [7]:
basket_df.describe()

,customer_id,product_id,basket_count
count,1.500000e+04,1.500000e+04,15000.000000
mean,1.808567e+07,3.269771e+07,2.153733
std,1.233000e+07,1.629455e+07,0.517929
min,4.784000e+03,4.939000e+04,2.000000
25%,8.659327e+06,3.137412e+07,2.000000
50%,1.520775e+07,3.694759e+07,2.000000
75%,2.663904e+07,4.502408e+07,2.000000
max,4.460824e+07,5.579097e+07,10.000000


### Check for Duplicates

In [8]:
print(f"Duplicate rows in customer_details: {customer_df.duplicated().sum()}")
print(f"Duplicate rows in basket_details: {basket_df.duplicated().sum()}")

Duplicate rows in customer_details: 0
Duplicate rows in basket_details: 0


### Check for Missing Values

In [9]:
print("Missing Values in Customer Details:")
print(customer_df.isnull().sum())
print("\n" + "="*60)
print("Missing Values in Basket Details:")
print(basket_df.isnull().sum())

Missing Values in Customer Details:
customer_id     0
sex             0
customer_age    0
tenure          0
dtype: int64

Missing Values in Basket Details:
customer_id     0
product_id      0
basket_date     0
basket_count    0
dtype: int64


### Unique Value Counts

In [10]:
print("Customer Details - Unique Values:")
for col in customer_df.columns:
    print(f"{col}: {customer_df[col].nunique()}")

Customer Details - Unique Values:
customer_id: 20000
sex: 4
customer_age: 93
tenure: 112


In [11]:
print("Basket Details - Unique Values:")
for col in basket_df.columns:
    print(f"{col}: {basket_df[col].nunique()}")

Basket Details - Unique Values:
customer_id: 13871
product_id: 13161
basket_date: 31
basket_count: 9


## Phase 3: Deep Dive Validation

Now that we've identified potential issues, let's validate them systematically before applying fixes.

### 1️⃣ Customer Age Validation

Check for invalid age values (negative or unrealistic ages).

In [12]:
# Check for invalid ages
total_customers = len(customer_df)

# Count customers with negative age
negative_age_count = (customer_df['customer_age'] < 0).sum()
negative_age_pct = (negative_age_count / total_customers) * 100

# Count customers with age > 120 (using 120 as the threshold for realistic human lifespan)
# Justification: The oldest verified human lived to 122 years, so 120 is a reasonable upper bound
unrealistic_age_count = (customer_df['customer_age'] > 120).sum()
unrealistic_age_pct = (unrealistic_age_count / total_customers) * 100

# Total invalid ages
total_invalid_age = negative_age_count + unrealistic_age_count
total_invalid_age_pct = (total_invalid_age / total_customers) * 100

print("=" * 60)
print("CUSTOMER AGE VALIDATION REPORT")
print("=" * 60)
print(f"\nTotal customers: {total_customers:,}")
print(f"\n❌ Negative ages (age < 0):")
print(f"   Count: {negative_age_count:,}")
print(f"   Percentage: {negative_age_pct:.2f}%")
print(f"\n❌ Unrealistic ages (age > 120):")
print(f"   Count: {unrealistic_age_count:,}")
print(f"   Percentage: {unrealistic_age_pct:.2f}%")
print(f"   Justification: 120 is used as the threshold since the oldest")
print(f"                  verified human lived to 122 years.")
print(f"\n📊 TOTAL INVALID AGES:")
print(f"   Count: {total_invalid_age:,}")
print(f"   Percentage: {total_invalid_age_pct:.2f}%")
print("=" * 60)

CUSTOMER AGE VALIDATION REPORT

Total customers: 20,000

❌ Negative ages (age < 0):
   Count: 1
   Percentage: 0.01%

❌ Unrealistic ages (age > 120):
   Count: 6,671
   Percentage: 33.36%
   Justification: 120 is used as the threshold since the oldest
                  verified human lived to 122 years.

📊 TOTAL INVALID AGES:
   Count: 6,672
   Percentage: 33.36%


### 2️⃣ Sex Column Validation

Inspect unique values and check for data quality issues.

In [13]:
# Inspect unique values in the sex column
print("=" * 60)
print("SEX COLUMN VALIDATION REPORT")
print("=" * 60)
print(f"\n📋 Unique values in 'sex' column:")
print(customer_df['sex'].unique())

print(f"\n📊 Value counts:")
print(customer_df['sex'].value_counts())

print(f"\n🔍 Value counts (including NaN):")
print(customer_df['sex'].value_counts(dropna=False))

# Check for potential issues
print(f"\n⚠️  POTENTIAL ISSUES DETECTED:")

# Check for casing inconsistencies
unique_values = customer_df['sex'].dropna().unique()
unique_lower = set([str(v).lower() for v in unique_values])
if len(unique_values) != len(unique_lower):
    print(f"   ❌ Casing inconsistencies found!")
    print(f"      Unique values: {list(unique_values)}")
else:
    print(f"   ✅ No casing inconsistencies detected")

# Check for unexpected categories (assuming M/F are expected)
expected_categories = {'M', 'F'}
unexpected = set(unique_values) - expected_categories
if unexpected:
    print(f"   ❌ Unexpected categories found: {unexpected}")
else:
    print(f"   ✅ All categories are expected (M/F)")

# Check for null values
null_count = customer_df['sex'].isnull().sum()
null_pct = (null_count / total_customers) * 100
print(f"\n   Missing values: {null_count:,} ({null_pct:.2f}%)")
print("=" * 60)

SEX COLUMN VALIDATION REPORT

📋 Unique values in 'sex' column:
['Male' 'Female' 'kvkktalepsilindi' 'UNKNOWN']

📊 Value counts:
sex
Male                15322
Female               4669
kvkktalepsilindi        8
UNKNOWN                 1
Name: count, dtype: int64

🔍 Value counts (including NaN):
sex
Male                15322
Female               4669
kvkktalepsilindi        8
UNKNOWN                 1
Name: count, dtype: int64

⚠️  POTENTIAL ISSUES DETECTED:
   ✅ No casing inconsistencies detected
   ❌ Unexpected categories found: {'Male', 'kvkktalepsilindi', 'UNKNOWN', 'Female'}

   Missing values: 0 (0.00%)


## Phase 4: Data Cleaning

Based on the validation findings, we now apply targeted cleaning operations.

### Clean customer_age

Replace invalid age values (< 0 or > 120) with NaN.

In [14]:
# Clean customer_age: Replace invalid values with NaN
# Count invalid values before cleaning
invalid_age_mask = (customer_df['customer_age'] < 0) | (customer_df['customer_age'] > 120)
invalid_count_before = invalid_age_mask.sum()

# Replace invalid ages with NaN
customer_df.loc[invalid_age_mask, 'customer_age'] = np.nan

# Count NaNs after cleaning
nan_count = customer_df['customer_age'].isnull().sum()
nan_percentage = (nan_count / len(customer_df)) * 100

print("=" * 60)
print("CUSTOMER AGE CLEANING RESULTS")
print("=" * 60)
print(f"Invalid values replaced: {invalid_count_before:,}")
print(f"Total NaN values in customer_age: {nan_count:,}")
print(f"Percentage of NaN values: {nan_percentage:.2f}%")
print("=" * 60)

CUSTOMER AGE CLEANING RESULTS
Invalid values replaced: 6,672
Total NaN values in customer_age: 6,672
Percentage of NaN values: 33.36%


### Clean sex

Map any value not in ["Male", "Female"] to "Unknown".

In [15]:
# Clean sex: Map any value not in ["Male", "Female"] to "Unknown"
# Define valid values
valid_sex_values = ["Male", "Female"]

# Count values before cleaning
print("=" * 60)
print("SEX COLUMN CLEANING RESULTS")
print("=" * 60)
print("\nBefore cleaning:")
print(customer_df['sex'].value_counts(dropna=False))

# Replace invalid values with "Unknown"
customer_df['sex'] = customer_df['sex'].apply(
    lambda x: x if x in valid_sex_values else "Unknown"
)

print("\n" + "-" * 60)
print("After cleaning:")
print(customer_df['sex'].value_counts(dropna=False))
print("=" * 60)

SEX COLUMN CLEANING RESULTS

Before cleaning:
sex
Male                15322
Female               4669
kvkktalepsilindi        8
UNKNOWN                 1
Name: count, dtype: int64

------------------------------------------------------------
After cleaning:
sex
Male       15322
Female      4669
Unknown        9
Name: count, dtype: int64


### Clean ID Columns in Basket Data

Convert ID columns to string type to prevent meaningless statistical operations.

In [16]:
# Convert ID columns to string type
# IDs should not be treated as numeric values
basket_df['product_id'] = basket_df['product_id'].astype(str)
basket_df['customer_id'] = basket_df['customer_id'].astype(str)

print("ID columns converted to string type:")
print(basket_df[['product_id', 'customer_id']].dtypes)

ID columns converted to string type:
product_id     object
customer_id    object
dtype: object


## Basket Data Integration Readiness

### Assessment Summary

Based on the validation checks performed above, here is the integration readiness assessment:

**✅ Strengths:**
- ID columns successfully converted to string type (prevents type mismatch during joins)
- Date range validation completed
- Referential integrity analysis completed

**⚠️ Risks Identified:**

1. **Orphaned Basket Records**: If orphaned customer_ids exist in the Basket table, these represent purchase transactions without corresponding customer profiles. This could result from:
   - Data collection issues
   - Deleted customer accounts
   - ID mismatches between systems

2. **Invalid Dates (NaT)**: Any NaT values in basket_date will prevent time-based analysis and may need to be excluded or imputed.

3. **Future Dates**: If detected, these represent data entry errors that could skew temporal analysis.

4. **Non-Purchasers**: Customers without purchase history may be:
   - Newly registered accounts
   - Inactive/churned customers
   - Test accounts

**🔧 Required Actions Before Integration:**

1. **Decision on Orphaned Records**: 
   - If orphan count > 0: Decide whether to drop orphaned basket records or investigate data source
   - Consider impact on revenue/transaction metrics

2. **Date Handling**:
   - Drop or impute NaT values based on business rules
   - Investigate and correct future dates

3. **Join Strategy**:
   - Use `LEFT JOIN` (basket → customer) to preserve all transactions while flagging orphans
   - Or use `INNER JOIN` to analyze only valid customer-basket pairs
   - Document excluded records for audit trail

**✅ Safe to Join If:**
- Orphan count is 0 or acceptable threshold (e.g., < 1%)
- NaT count is minimal (< 5%)
- No future dates detected
- Business rules for handling non-purchasers are defined

**Recommendation**: Review the specific counts above before proceeding with data integration.

In [17]:
# Get unique customer IDs from both datasets
basket_customer_ids = set(basket_df['customer_id'].unique())
customer_customer_ids = set(customer_df['customer_id'].astype(str).unique())

# Calculate orphaned records (basket customers not in customer table)
orphaned_ids = basket_customer_ids - customer_customer_ids
orphan_count = len(orphaned_ids)
orphan_records = basket_df[basket_df['customer_id'].isin(orphaned_ids)].shape[0]
orphan_pct = (orphan_records / len(basket_df)) * 100

# Calculate non-purchasers (customers not in basket table)
non_purchaser_ids = customer_customer_ids - basket_customer_ids
non_purchaser_count = len(non_purchaser_ids)
non_purchaser_pct = (non_purchaser_count / len(customer_df)) * 100

print("=" * 60)
print("JOIN INTEGRITY CHECK REPORT")
print("=" * 60)
print(f"\n📊 Dataset Overview:")
print(f"  Unique customers in Customer table: {len(customer_customer_ids):,}")
print(f"  Unique customers in Basket table: {len(basket_customer_ids):,}")

print(f"\n⚠️  Orphaned Records (Basket → Customer):")
print(f"  Unique customer_ids in Basket NOT in Customer: {orphan_count:,}")
print(f"  Total orphaned basket records: {orphan_records:,} ({orphan_pct:.2f}%)")
if orphan_count > 0:
    print(f"   ❌ WARNING: Foreign key violations detected")
else:
    print(f"   ✅ No orphaned records")

print(f"\n📉 Non-Purchasers (Customer → Basket):")
print(f"  Unique customer_ids in Customer NOT in Basket: {non_purchaser_count:,} ({non_purchaser_pct:.2f}%)")
if non_purchaser_count > 0:
    print(f"   ℹ️  INFO: These customers have no purchase history")
else:
    print(f"   ✅ All customers have purchase records")

print("=" * 60)

JOIN INTEGRITY CHECK REPORT

📊 Dataset Overview:
  Unique customers in Customer table: 20,000
  Unique customers in Basket table: 13,871

⚠️  Orphaned Records (Basket → Customer):
  Unique customer_ids in Basket NOT in Customer: 13,807
  Total orphaned basket records: 14,928 (99.52%)
   ❌ WARNING: Foreign key violations detected

📉 Non-Purchasers (Customer → Basket):
  Unique customer_ids in Customer NOT in Basket: 19,936 (99.68%)
   ℹ️  INFO: These customers have no purchase history


### Join Integrity Check

Assess referential integrity between Customer and Basket datasets.

In [18]:
# Convert basket_date to datetime with error handling
basket_df['basket_date'] = pd.to_datetime(basket_df['basket_date'], errors='coerce')

# Count invalid dates (NaT)
nat_count = basket_df['basket_date'].isna().sum()
nat_percentage = (nat_count / len(basket_df)) * 100

# Get earliest and latest dates (excluding NaT)
earliest_date = basket_df['basket_date'].min()
latest_date = basket_df['basket_date'].max()

# Check for future dates (beyond today)
today = pd.Timestamp.now()
future_dates_count = (basket_df['basket_date'] > today).sum()

print("=" * 60)
print("BASKET DATE VALIDATION REPORT")
print("=" * 60)
print(f"\nTotal basket records: {len(basket_df):,}")
print(f"\nInvalid dates (NaT): {nat_count:,} ({nat_percentage:.2f}%)")
print(f"\nDate Range:")
print(f"  Earliest date: {earliest_date}")
print(f"  Latest date: {latest_date}")
print(f"\n⚠️  Future dates detected: {future_dates_count:,}")
if future_dates_count > 0:
    print(f"   ❌ WARNING: Basket contains dates beyond today ({today.date()})")
else:
    print(f"   ✅ No future dates detected")
print("=" * 60)

BASKET DATE VALIDATION REPORT

Total basket records: 15,000

Invalid dates (NaT): 0 (0.00%)

Date Range:
  Earliest date: 2019-05-20 00:00:00
  Latest date: 2019-06-19 00:00:00

⚠️  Future dates detected: 0
   ✅ No future dates detected


### Validate basket_date

Convert basket_date to datetime and check for invalid dates.

## Phase 5: Post-Cleaning Verification

Verify that cleaning operations were successful.

### Customer Details: Post-Cleaning Summary

In [19]:
print("Customer DataFrame Info After Cleaning:")
customer_df.info()
print("\n" + "="*60)
print("Customer DataFrame Description After Cleaning:")
customer_df.describe()

Customer DataFrame Info After Cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   customer_id   20000 non-null  int64  
 1   sex           20000 non-null  object 
 2   customer_age  13328 non-null  float64
 3   tenure        20000 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 625.1+ KB

Customer DataFrame Description After Cleaning:


,customer_id,customer_age,tenure
count,2.000000e+04,13328.000000,20000.000000
mean,1.760040e+07,32.429622,44.396800
std,8.679505e+06,11.605978,31.998376
min,2.093000e+03,3.000000,4.000000
25%,1.188115e+07,27.000000,21.000000
50%,1.560912e+07,31.000000,35.000000
75%,2.228484e+07,38.000000,60.000000
max,4.462566e+07,120.000000,133.000000


### Basket Details: Post-Cleaning Summary

In [20]:
print("Basket DataFrame Info After Cleaning:")
basket_df.info()
print("\n" + "="*60)
print("Basket DataFrame Description After Cleaning:")
basket_df.describe()

Basket DataFrame Info After Cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   customer_id   15000 non-null  object        
 1   product_id    15000 non-null  object        
 2   basket_date   15000 non-null  datetime64[ns]
 3   basket_count  15000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 468.9+ KB

Basket DataFrame Description After Cleaning:


,basket_date,basket_count
count,15000,15000.000000
mean,2019-06-01 03:44:09.600000,2.153733
min,2019-05-20 00:00:00,2.000000
25%,2019-05-26 00:00:00,2.000000
50%,2019-05-28 00:00:00,2.000000
75%,2019-06-09 00:00:00,2.000000
max,2019-06-19 00:00:00,10.000000
std,NaN,0.517929


### Final Data Quality Summary

In [21]:
print("="*60)
print("FINAL DATA QUALITY SUMMARY")
print("="*60)
print(f"\nCustomer Dataset:")
print(f"  Total records: {len(customer_df):,}")
print(f"  Duplicates: {customer_df.duplicated().sum()}")
print(f"  Missing ages: {customer_df['customer_age'].isnull().sum()} ({(customer_df['customer_age'].isnull().sum()/len(customer_df)*100):.2f}%)")
print(f"  Unknown sex values: {(customer_df['sex'] == 'Unknown').sum()}")

print(f"\nBasket Dataset:")
print(f"  Total records: {len(basket_df):,}")
print(f"  Duplicates: {basket_df.duplicated().sum()}")
print(f"  ID columns converted to string: Yes")
print("="*60)

FINAL DATA QUALITY SUMMARY

Customer Dataset:
  Total records: 20,000
  Duplicates: 0
  Missing ages: 6672 (33.36%)
  Unknown sex values: 9

Basket Dataset:
  Total records: 15,000
  Duplicates: 0
  ID columns converted to string: Yes


## Data Integration Decision & Final Notes

After thorough validation and integrity testing, the join between Customer and Basket datasets revealed approximately 99% ID mismatch. This represents a fundamental structural limitation in how these datasets were collected or linked, rather than a simple data quality issue that can be resolved through cleaning.

**Decision**: We will **NOT** force a join or drop mismatched records. Instead, these datasets will be analyzed as **separate analytical views**:
- **Customer Demographics Analysis**: Age distribution, gender patterns, tenure analysis
- **Transaction Patterns Analysis**: Purchase frequency, product performance, temporal trends

This approach preserves data correctness and analytical integrity. Forcing a join with such high mismatch rates would produce misleading insights and violate fundamental data quality principles. By acknowledging this limitation transparently, we ensure that downstream analysis remains trustworthy and defensible.

**Conclusion**: This case study demonstrates proper data validation, cleaning, and the critical importance of making honest decisions when data integration is not feasible. Separate analysis is always preferable to fabricated relationships.

---